In [ ]:
from pathlib import Path
import sys

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
## Enable access to modules in project directory
proj_root = (Path('..') / 'app').resolve()
if proj_root not in sys.path:
    sys.path.append(proj_root)


In [ ]:

data = pd.read_csv(proj_root / 'data' / 'happiness.csv', delimiter=',', header=0, skip_blank_lines=False)

## Data Cleaning

In [ ]:
#Select the quantatitive columns. We are ignoring categorical data for this analysis
cleaned_featureset = data.select_dtypes(include=['int64', 'float64'])

#Remove happiness_rank and happiness_score
cleaned_featureset = cleaned_featureset.drop(columns=['happiness_rank', 'happiness_score'])

#Convert 0's to NAN as there shouldn't be 0's in this dataset unless the value couldn't be calculated
cleaned_featureset = cleaned_featureset.replace(0.0, np.NaN)

#Replace the NA's with mean of column to keep consistent length
cleaned_featureset = cleaned_featureset.apply(lambda x: x.fillna(x.mean()),axis=0)

#Add constant to columns with negative values so that all are positive
cleaned_featureset = cleaned_featureset.apply(
    lambda x: x.sub(x.min()-1) if x.min() < 0 else x)

#Transform columns which have significant outliers and skewed data
#Chose not to remove outliers as their information could still be significant
transform_cube_cols = ['cellular_subscriptions', 'familiy_income_gini_coeff']
transform_log_cols=['military_expenditures[%]', 'population', 'GDP_per_capita[$]', 'inflation_rate[%]']

cleaned_featureset[transform_cube_cols] = cleaned_featureset[
    transform_cube_cols].apply(lambda x: x.transform(np.cbrt))

cleaned_featureset[transform_log_cols] = cleaned_featureset[
    transform_log_cols].apply(lambda x: x.transform(np.log))
cleaned_featureset['surplus_deficit_GDP[%]'] = cleaned_featureset['surplus_deficit_GDP[%]'].pow(3)
#normalize data to fit between 0 and 1
cleaned_featureset = cleaned_featureset.apply(
    lambda x: [(el-x.min())/(x.max()-x.min()) for el in x], axis=0)

#Nooow lets have a look at the sexy data | or not because latex outputs to PDF in ugly way...


In [ ]:
cleaned_featureset.describe()

We can see that some of this data is still slightly skewed but the standardisation and transformation has done a pretty effective job. 

In [ ]:
#Adding the happiness score back into the featureset.
cleaned_featureset.insert(0, 'happiness_score', data['happiness_score'])

# Summary of Scatterplots
The data from many of these scatterplots have already been normalized. Many of them reflect the contribution this study found in the variable's effect on the happiness score. The greater the score, the greater the effect on the survey respondents effect of happiness. 

## Economy

In [ ]:
plot_w = 3
plot_h = 5
fig, ax = plt.subplots(plot_h, plot_w, sharey="row")
fig.set_figheight(20)
fig.set_figwidth(30)
for i in range(plot_h):
    for j in range(plot_w):
        feature = cleaned_featureset.columns[1+i*(plot_w)+j]
        ax[i,j].scatter(cleaned_featureset[feature], cleaned_featureset['happiness_score'])
        ax[i,j].set_title(feature)
plt.show()

## Summary
We can see a range of correlations between features in our dataset and our happiness score. The model will use those deemed as most significant. The strongest features economy, family, health, GDP_per_capita and internet access population.
There is a weaker correlation with the happiness score and inflation rate, cellular subscriptions and dystopia residual however it may be enough to improve our model, with an appropriate weighting assigned by the algorithm.

It would seem intuitive that economy, family and health are all signficant as a thriving economy, people with family and a sense of belonging and a healthly state of being could improve a sense of happiness. A strong GDP per capita might keep people busy and an economy productive and internet access enable people to be connected to others as well as information. Counter to this, economic prosperity and higher GDP oer capita could result in higher levels of stressand higher suicide rates etc, High numbers of family could be due to high mortality in children, or requirements of support in farming (in third world countries) and internet can be used to spread disinformation and control populations. It is important to note that these studies are observational, surveys which are littered with personal bias. The factors for many of these variables are also abstracted and give us little insight into how they were calculated/weighted.

# Q2.2

Load data and set up packages

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn import metrics

## Linear model

In [ ]:
#This is the featureset that I am choosing to use.
featureset_keys = ['economy', 'family', 'health', 
                   'freedom', 'dystopia_residual', 
                   'internet_access_population[%]', 
                   'cellular_subscriptions', 'GDP_per_capita[$]', 
                   'inflation_rate[%]'
                  ]
featureset = cleaned_featureset[featureset_keys]


In [ ]:
#Showing a correlation matrix from the featureset against the happiness score.
plt.matshow(cleaned_featureset.corr())
cleaned_featureset.corr()['happiness_score'].sort_values(ascending=False)


In [ ]:
import pickle

In [ ]:
#KERNEL RIDGE LINEAR REGRESSION
def train_model(df, featureset_keys, kernel="linear", alpha=1.0, gamma=None, degree=None, coef0=None):

    # Setup Parameters for Model
    kr_args = {"kernel": kernel, "alpha": alpha}

    # Validate parameters for polynomial
    if kernel == "polynomial":
        if degree is None or coef0 is None:
            print("Must provide a parameter for degree and coef0")
            return None
        else:
            kr_args["gamma"] = gamma
            kr_args["degree"] = degree
            kr_args["coef0"] = coef0
    
    plt.figure(figsize=(20,10))
    # Store the results of each training run
    predictions = []
    scores = []
    
    # Save the best model to return
    best_model = None
    baseline = 0.0
    
    i = 0
    for train, test in RepeatedKFold(n_splits=5, n_repeats=1).split(df):
        
        # Split dataset
        train_x, train_y = (df.iloc[train])[featureset_keys], (df.iloc[train])['happiness_score']
        test_x, test_y =(df.iloc[test])[featureset_keys], (df.iloc[test])['happiness_score']
        
        # Initialise model
        kr_model = KernelRidge(**kr_args)
        
        # Train model
        kr_model.fit(train_x, train_y)
        
        # Evaluate model 
        pred_y = kr_model.predict(test_x)
        score = kr_model.score(test_x, test_y)
        
        # Save if better then previous
        if score > baseline:
            best_model = kr_model
        predictions.append(pred_y)
        plt.scatter(test_y, p, label=f"iter {i}")
        scores.append(score)
        i = i+1
        
    plt.plot(df['happiness_score'], df['happiness_score'], label='actual')
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend(loc="upper left", bbox_to_anchor=(1.05, 1))
    return best_model, plt, scores
    print(pred_scores)
    
# best_model, plt, scores = train_model(cleaned_featureset, featureset_keys)
best_model, plt, scores = train_model(cleaned_featureset, featureset_keys, kernel="polynomial", degree=3, coef0=1)

plt

In [ ]:

filename = 'kkr_model.pkl'
pickle.dump(best_model, open(proj_root / 'model' / filename, 'wb'))

In [ ]:
# Pull serialized model
loaded_model = pickle.load(open(proj_root / 'model' / filename, 'rb'))
# generate predictions
# economy, family, health, freedom, dystop., internet access, cellular subs, gdp, inflation
result = loaded_model.predict([[1,1,1,1,1,1,1,1,1]])
# Add prediction to db
post_score(input, result, actual = None)
print(result)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
train_x, train_y = (cleaned_featureset.iloc[train])[featureset_keys], (cleaned_featureset.iloc[train])['happiness_score']
components = pca.fit_transform(train_x)

# If we wan to plot where all the countries lie in a PCA
plt.scatter(components[:,0], components[:,1])

### Linear Results
Running the Kernel Ridge model with a 1.2 alpha and using 5 fold cross validation, R^2 averaged at 0.95 plus/minus 0.01 in subsequent runs. The model runs with strong accuracy. The variation is mostly present in the lower happiness scores values. This could be due to less data being available or a lesser weight being placed on the available data that better predicts those values.

## Quadratic model

In [ ]:
results = pd.DataFrame(columns=["cross_validation_results"])
def train_model(df, features, alpha=1.0, kernel="linear", gamma=None, degree=None, coef0=None):
    
    # Setup parameters for model
    kr_args = {"kernel": kernel, "alpha": alpha, "gamma": gamma}        
    
    if kernel == "polynomial":
        kr_args["degree"] = degree 
        kr_args["coef0"] = coef0
    
    
    
    # Create and run cross validation
    pred_list = []
    pred_scores = []
    for train, test in RepeatedKFold(n_splits=5, n_repeats=1).split(df):
        
        train_x, train_y = df.iloc[train][features], df.iloc[train]['happiness_score']
        test_x, test_y = df.iloc[test][features], df.iloc[test]['happiness_score']
        
        print(train_x, test_x)
        print(train_y, test_y)
        # Initialise
        kr_model = KernelRidge(**kr_args)
        
        # Train
        fit = kr_model.fit(train_x, train_y)
        
        # Predict 
        prediction = kr_model.predict(test_x)
        pred_list.append(prediction)
        
        # Evaluate
        score = kr_model.score(test_x, test_y)
        print(f"Score for {kernel} model is {score}")
        pred_scores.append(score)
        
    for p in pred_list_quad:
        plt.scatter(test_y, p)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        
    print(f'Predicted value average across 5 runs are {sum(pred_val_list_quad)/len(pred_val_list_quad):.4f}')
    

In [ ]:
train_model(cleaned_featureset, featureset_keys, "linear")

In [ ]:
# KERNEL RIDGE POLYNOMIAL REGRESSION

rkf = RepeatedKFold(n_splits=5, n_repeats=1)
pred_list_quad = []
pred_val_list_quad = []
for train, test in rkf.split(cleaned_featureset):
    train_x, train_y = (cleaned_featureset.iloc[train])[featureset_keys],(cleaned_featureset.iloc[train])['happiness_score']
    test_x, test_y = (cleaned_featureset.iloc[test])[featureset_keys], (cleaned_featureset.iloc[test])['happiness_score']
    #Create model
    clf_quad = KernelRidge(alpha=1.0, kernel='polynomial', gamma=1.3, degree=2, coef0=1.3)
    #Build model with training data
    fit_quad = clf_quad.fit(train_x, train_y)
    #now use the model to 
    prediction = clf_quad.predict(test_x)
    pred_list_quad.append(prediction)
    score = clf_quad.score(test_x, test_y)
    pred_val_list_quad.append(score)

for p in pred_list_quad:
    plt.scatter(test_y, p)

plt.xlabel('True Values')
plt.ylabel('Predictions')

print('Predicted value average across 5 runs are {0:.4f}'.format(sum(pred_val_list_quad)/len(pred_val_list_quad)))

### Quadratic Results
The algorithm used the Kernel Ridge polynomial model with alpha of 1.0, gamma of 1.3 and degrees of 2 (quadratic). R^2 across 5 fold cross validation averaged at 0.96 plus/minus 0.01 in subsequent runs. Like the linear model, the highest variance in the prediction occurred in the lower happiness scores. The model overpredicted earlier results initially, however varied between an over and underprediction. It was most accurate, and least varied around the mean and median of the happiness score which makes sense, as that is where most of the data lies.

## Gaussian model

In [ ]:
#KERNEL RIDGE GAUSSIAN REGRESSION

rkf_g = RepeatedKFold(n_splits=5, n_repeats=1)
pred_list_g = []
pred_val_list_g = []
for train, test in rkf_g.split(cleaned_featureset):
    train_x, train_y = (cleaned_featureset.iloc[train])[featureset_keys],(cleaned_featureset.iloc[train])['happiness_score']
    test_x, test_y = (cleaned_featureset.iloc[test])[featureset_keys], (cleaned_featureset.iloc[test])['happiness_score']
    #Create model
    clf = KernelRidge(alpha=.01, kernel='rbf', gamma=0.2)
    #Build model with training data
    fit = clf.fit(train_x, train_y)
    #now use the model to 
    p = clf.predict(test_x)
    pred_list_g.append(p)
    s = clf.score(test_x, test_y)
    pred_val_list_g.append(s)

for p in pred_list_g:
    plt.scatter(test_y, p)

plt.xlabel('True Values')
plt.ylabel('Predictions')

print('Predicted value average across 5 runs are {0:.4f}'.format(sum(pred_val_list_g)/len(pred_val_list_g)))

The Gaussian algorithm used the Kernel Ridge radial basis function model with an alpha of 0.01 and gamma of 0.2. The $R^2$ average across 5 fold cross validation was 0.97 plus/minus 0.01. The model was most accurate using the gaussian model, likely because of the pre-processing work to normalize the data. Like the other two models, the highest variance occurred at the lowest happiness scores.

## Comparison

The Gaussian model runs best ouut of the 3. This is likely because I normalized the data during pre-processing. Compared to my colleagues I am disappointed I didn't get a more accurate model however I think that this is within the range of accepatble measures and I would argue that the offset from truth would be because of a better generalization. However it is difficult to justify this model in any real cases due to the lack of quality of the data. I believe that happiness is such a subjective measure that any organisation building data on such a topic can only result in failure. I therefore urge anyone reading this report to be careful when referencing. I also urge the reader to seek other literature as there are bodies of longitudinal studies that challenge the null hypothesis that certain factors such as GDP are positively related to happiness. Whilst saying this, I would like to articulate that this opinion is outside of the scope of this report.